### Cluster Test

In [ ]:
"""
Author: Kye D Nichols
This script runs clustering methods
"""

In [ ]:
import os, pickle
import pandas as pd
import numpy as np
from src.helper_scripts import *
from kmodes.kprototypes import KPrototypes
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances
from sklearn_extra.cluster import KMedoids
from sklearn.metrics.pairwise import pairwise_distances
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.metrics import rand_score
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.offline import iplot, plot
from sklearn.metrics import silhouette_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.manifold import TSNE
import umap
import json
from sparsemedoid import clustering

In [ ]:

# number of clusters (int), list of distance types ([str]), ...
# from J's run.py code in test subdirectory
def run_kmedoids_clustering(clusters, distance_types, normalization_param, X_df):
    total_runs = (len(clusters) * len(distance_types) * len(normalization_param))
    Scores = np.zeros((1, total_runs))
    barcodes = X_df.index.to_list()
    X = X_df.to_numpy()
    P = X.shape[0]
    N = X.shape[1]
    prefix_cols = []
    all_feature_weights = np.zeros((N, total_runs))
    all_cluster_labels = np.zeros((P, total_runs))
    iter1 = 0
    for K in clusters:
        for distance in distance_types:
            for S in normalization_param:
                results_path_prefix = f"K={K}_dist={distance}_S={S}"
                prefix_col = f"N={N}_K={K}_dist={distance}_nparam={S}"
                prefix_cols.append(results_path_prefix)
                (
                    cluster_labels,
                    feature_weights,
                    feature_order,
                    weighted_distances,
                ) = clustering.sparse_kmedoids(
                    X,
                    distance_type=distance,
                    k=K,
                    s=S,
                    max_attempts=6,
                    method="pam",
                    init="build",
                    max_iter=100,
                    random_state=None,
                )
                Scores[0, iter1] += silhouette_score(
                    weighted_distances, cluster_labels, metric="precomputed"
                )
                all_feature_weights[:, iter1] = feature_weights
                all_cluster_labels[:, iter1] = cluster_labels
                iter1 += 1
    feature_weights_df = pd.DataFrame(all_feature_weights)
    cluster_labels_df = pd.DataFrame(all_cluster_labels)
    cluster_labels_df.index = barcodes
    cluster_labels_df.columns = prefix_cols
    feature_weights_df.index = X_df.columns.to_list()
    feature_weights_df.columns = prefix_cols
    scores_df = pd.DataFrame(Scores)
    scores_df.columns = prefix_cols
    return scores_df, cluster_labels_df, feature_weights_df

def gower_distance(X):
    gower_matrix = pairwise_distances(X, metric='manhattan')
    return gower_matrix

def run_skmedoids(indf, cluster_num, tag, output_dir):
    scores_df, cluster_labels_df, feature_weights_df = run_kmedoids_clustering([cluster_num],
                                                                                distance_types,
                                                                                norm_params,
                                                                                indf)
    scores_df.to_csv(os.path.join(output_dir, "%s_%s_K=%i.csv" % (tag, kmedoid_score_str, cluster_num)))
    feature_weights_df.to_csv(os.path.join(output_dir, "%s_%s_K=%i.csv" % (tag, kmedoid_weight_str, cluster_num)))
    cluster_labels_df.to_csv(os.path.join(output_dir, "%s_%s_K=%i.csv" % (tag, kmedoid_lbl_str, cluster_num)))
    indf.to_csv(os.path.join(output_dir, "%s_%s_K=%i.csv" % (tag, "input", cluster_num)))

def run_vanilla_kmedoids_clustering(indf, cluster_num, tag, output_dir):
    gower_matrix = gower_distance(np.array(indf))
    kmedoids_gower = KMedoids(n_clusters=cluster_num, metric='precomputed').fit(gower_matrix)
    clusters_gower = kmedoids_gower.labels_

    kmedoids_euc = KMedoids(n_clusters=cluster_num, metric='euclidean').fit(np.array(indf))
    clusters_euc = kmedoids_euc.labels_
    indf['Cluster_Gower'] = kmedoids_gower.labels_
    indf['Cluster_Euc'] = kmedoids_euc.labels_
    out_path = os.path.join(output_dir, "%s_%s_K=%i.csv" % (tag, "Kmedoids", cluster_num))
    indf.to_csv(out_path)

def run_kprototype_clustering(indf, cluster_num, tag, output_dir, cat_col_idx):
    kproto = KPrototypes(n_clusters=cluster_num, init='Cao', verbose=2)
    clusters = kproto.fit_predict(np.array(indf), categorical=cat_col_idx)
    indf['Cluster'] = clusters
    out_path = os.path.join(output_dir, "%s_%s_K=%i.csv" % (tag, "Kprototype", cluster_num))
    indf.to_csv(out_path)

def run_kmeans_clustering(indf, cluster_num, tag, output_dir):
    kmeans = KMeans(n_clusters=cluster_num)
    clusters = kmeans.fit_predict(indf.values)
    indf['Cluster'] = clusters
    out_path = os.path.join(output_dir, "%s_%s_K=%i.csv" % (tag, "KMeans", cluster_num))
    indf.to_csv(out_path)
    
def run_agglo_clustering(indf, cluster_num, tag, output_dir):
    agglo = AgglomerativeClustering(n_clusters=cluster_num)
    clusters = agglo.fit_predict(np.array(indf))
    indf['Cluster'] = clusters
    out_path = os.path.join(output_dir, "%s_%s_K=%i.csv" % (tag, "Agglo", cluster_num))
    indf.to_csv(out_path)

   

In [ ]:
my_tag = "TCGA-STAD-Test"
#embedding_type = "CustOmics"
embedding_type = "MixOmics"

encoding_col = "Subtype_Selected"
omics_pickle_path = "integrated/TCGA-STAD-Test.pickle"
omics_df = pickle.load(open(omics_pickle_path,'rb'))
Mixomics_embedding_path = "integrated/embeddings/TCGA-STAD-Test_MixOmics_Embedding.csv"
Customics_embedding_path = "integrated/embeddings/TCGA-STAD-Test_Customics_latent.csv"

clinical_path = "downloads/TCGA-STAD/STAD_clinical.csv"
sample_list_path = "integrated/TCGA-STAD-Test-sample_list.txt"
config_path= "src/TCGA_Config-DS.json"
ground_truth_path = "integrated/TCGA-STAD-Test_labels.csv"
results_dir = "results"

In [ ]:
config_json = json.load(open(config_path, 'r'))
datatype_tag_dict = config_json['datatypes']
encodings = config_json['encodings']
encoding = encodings[encoding_col]
cat_cols = config_json['categorical']
num_cols = config_json['numerical']
select_cols=cat_cols+num_cols
mysamples = [i.strip() for i in open(sample_list_path, 'r').readlines()]

In [ ]:
clinical_df = pd.read_csv(clinical_path, index_col=2)
clinical_df = clinical_df.loc[mysamples][select_cols].dropna(axis="rows")
gt_df = pd.read_csv(ground_truth_path, index_col=0)

In [ ]:
customics_df = pd.read_csv(Customics_embedding_path, index_col=0)
mixomics_df= pd.read_csv(Mixomics_embedding_path, index_col=0)

In [ ]:
if embedding_type ==  "CustOmics": embedding_df = customics_df
else: embedding_df = mixomics_df

In [ ]:
clinical_df_enc = one_hot_encode(clinical_df, cat_cols)
clinical_df_enc_norm = one_hot_encode(clinical_df, cat_cols, norm=True)

In [ ]:
kmedoid_score_str = "skmedoids_scores"
kmedoid_weight_str = "skmedoids_feature_weights"
kmedoid_lbl_str = "skmedoids_cluster_labels"
distance_types = ["gower", "wishart", "podani"]
norm_params = [1.01]+[i/10 for i in range(11,45,1)]
cluster_num = 4 

In [ ]:
#all_df = embedding_df.loc[mysamples].join(clinical_df.loc[mysamples])
all_df = embedding_df.join(clinical_df).dropna(axis="rows")
all_df_enc = embedding_df.join(clinical_df_enc).dropna(axis="rows")
all_df_enc_norm = embedding_df.join(clinical_df_enc_norm).dropna(axis="rows")

In [ ]:
run_skmedoids(all_df, cluster_num, "%s-%s"%(my_tag, embedding_type), results_dir)

#list1 = all_df_enc.columns.to_list()
#cat_col_idx = [index for index, item in enumerate(list1) if item in cat_cols]
#run_kprototype_clustering(all_df_enc, cluster_num, mytag, results_dir, cat_col_idx)

#run_vanilla_kmedoids_clustering(all_df_enc, cluster_num, mytag, results_dir)
#run_kmeans_clustering(all_df_enc_norm, cluster_num, mytag, results_dir)
#run_agglo_clustering(all_df_enc_norm, cluster_num, mytag, results_dir)

### Create graphs 

In [ ]:
skmedoids_labels_path = os.path.join(results_dir, "%s-%s_skmedoids_cluster_labels_K=4.csv" %(my_tag, embedding_type))
skmedoids_weights_path = os.path.join(results_dir, "%s-%s_skmedoids_feature_weights_K=4.csv"%(my_tag, embedding_type))
skmedoids_scores_path = os.path.join(results_dir, "%s-%s_skmedoids_scores_K=4.csv"%(my_tag, embedding_type))
skmedoids_strs = ["K=4_dist=gower", "K=4_dist=wishart", "K=4_dist=podani"]

In [ ]:
skmedoids_labels = pd.read_csv(skmedoids_labels_path, index_col=0)
skmedoids_weights = pd.read_csv(skmedoids_weights_path, index_col=0)
skmedoids_scores = pd.read_csv(skmedoids_scores_path, index_col=0)

In [ ]:
# plot heatmap of norm weights
def plot_heatmap(title, weights_df, id_strs, height=1200):
    select_cols = []
    for id_str in id_strs:
        for col in weights_df.columns:
            if id_str in col:
                select_cols.append(col)

    layout = go.Layout(width=1000, height=height)
    data = go.Heatmap(z=np.array(weights_df[select_cols]),
                     x=[i for i in weights_df[select_cols].columns.to_list()],
                     y=weights_df[select_cols].index.to_list(),
                     colorscale = 'Blues')
    fig = go.Figure(data=[data], layout=layout)
    fig.update_layout(yaxis = dict(tickfont = dict(size=15)))
    fig.update_layout(title = title)
    fig.update_layout(legend = dict(font = dict(size=15)))
    fig.update_layout(title = dict(font = dict(size=17)))
    return fig

In [ ]:
skmedoids_norm_weights = norm_feature_weights(skmedoids_weights)

In [ ]:
fig_hm = plot_heatmap(my_tag, skmedoids_norm_weights, skmedoids_strs)
#plot(fig_hm, filename=os.path.join(results_dir,"K=%s_%s_Feature_Map.html" % (cluster_num, my_tag)))
fig_hm

In [ ]:
# plot score
def plot_score(title, y_axis_str, id_strs, scores_df, norm_param, font_size = 20, width=750, height=400):
    plot_dict= {"norm param":norm_param}; ys = []
    for id_str in id_strs:
        select_cols = []
        for col in scores_df.columns:
            if id_str in col:
                select_cols.append(col)
        y_str = "%s " % id_str
        ys.append(y_str)
        plot_dict[y_str] = scores_df[select_cols].loc[0].to_list()
    score_plot_df = pd.DataFrame().from_dict(plot_dict)
    fig = px.line(score_plot_df, x="norm param", y=ys, width=width, height=height)
    fig.update_layout(yaxis = dict(tickfont = dict(size=font_size)))
    fig.update_layout(xaxis = dict(tickfont = dict(size=font_size)))
    fig.update_layout(xaxis_title="Normalization Parameter", yaxis_title=y_axis_str)
    fig.update_layout(yaxis_title = dict(font = dict(size=font_size)))
    fig.update_layout(xaxis_title = dict(font = dict(size=font_size)))
    fig.update_layout(title = title)
    fig.update_layout(legend = dict(font = dict(size=font_size-5)))
    fig.update_layout(title = dict(font = dict(size=font_size+2)))
    fig.update_traces(line={'width': 6})
    fig.write_image(os.path.join(results_dir, "%s_%s.png" % (y_axis_str , my_tag)))
    return fig

In [ ]:
num_nonz_wieghts = get_perc_nonzero(skmedoids_weights)
plot_score(my_tag, "Non-Zero Features", skmedoids_strs, num_nonz_wieghts, norm_params)

In [ ]:
plot_score(my_tag, "Silouette Score",skmedoids_strs, skmedoids_scores, norm_params)

In [ ]:
def runtsne(indf, imgdir, tag, labels, labels_key="labels", save_img=True, encoding=None):
    labels_temp = labels.copy()
    if encoding:
        encoding_rev = {encoding[k]:k for k in list(encoding)}
        labels_temp[labels_key] = labels_temp[labels_key].map(lambda s: encoding_rev.get(s))
    X = np.array(indf)
    X_embedded = TSNE(n_components = 2,
                      learning_rate = 'auto',
                      init='random',
                      perplexity = 30).fit_transform(X)
    tsne_df = pd.DataFrame({'tsne_1': X_embedded[:,0],
                            'tsne_2': X_embedded[:,1],
                            'label': labels_temp[labels_key]})
    tsne_df.index = indf.index
    fig = px.scatter(tsne_df, x="tsne_1", y="tsne_2", color='label', width=500, height=400)
    fig.write_image(os.path.join(imgdir, "%s_tnse.png" % tag))
    return fig

def runumap(indf, imgdir, tag, labels, labels_key="labels", save_img=True, encoding=None):
    labels_temp = labels.copy()
    if encoding:
        encoding_rev = {encoding[k]:k for k in list(encoding)}
        labels_temp[labels_key] = labels_temp[labels_key].map(lambda s: encoding_rev.get(s))
    X = np.array(indf)
    X_embedded = umap.UMAP(n_neighbors=10,
                      min_dist=0.3,
                      metric='correlation').fit_transform(X)
    umap_df = pd.DataFrame({'umap_1': X_embedded[:,0],
                            'umap_2': X_embedded[:,1],
                            'label': labels_temp[labels_key]})
    umap_df.index = indf.index
    fig = px.scatter(umap_df, x="umap_1", y="umap_2", color='label', width=500, height=400)
    fig.write_image(os.path.join(imgdir, "%s_umap.png" % tag))
    return fig


In [ ]:
#runtsne(embedding_df, results_dir, my_tag, gt_df)

In [ ]:
#runumap(embedding_df, results_dir, my_tag, gt_df)

In [ ]:
# calculate adjusted and unadjusted Rand scores
def get_rand_index(cluster_df, lbls):
    out_dict_rand={}; out_dict_arand={}
    for col in cluster_df.columns:
        out_dict_rand[col]=[rand_score(cluster_df[col], lbls)]
        out_dict_arand[col]=[adjusted_rand_score(cluster_df[col], lbls)]
    return pd.DataFrame().from_dict(out_dict_rand), pd.DataFrame().from_dict(out_dict_arand)

# plot Rand Index or adjusted
def plot_rand(tag, id_strs, scores_df, norm_param, lbls):
    rand_df, arand_df = get_rand_index(skmedoids_labels, lbls)
    fig1 = plot_score(my_tag+"_ARand", "Adj. Rand", id_strs, arand_df, norm_param)
    fig2 = plot_score(my_tag+"_Rand", "Rand", id_strs, rand_df, norm_param)
    return fig1, fig2


In [ ]:
lbls = gt_df.loc[skmedoids_labels.index.to_list()].sort_index(ascending=False)["labels"]
arand_fig, rand_fig = plot_rand(my_tag, skmedoids_strs, skmedoids_labels , norm_params, lbls)

In [ ]:
rand_fig

In [ ]:
arand_fig

In [ ]:
def make_tsne_mixed(dist_types, norm_weights, input_df, lbl_list, norm_list):
    figs = []
    for dist_type in dist_types:
        for sn in norm_list:
            weighted_df = input_df.copy()
            wts = norm_weights["K=4_dist=%s_S=%s"% (dist_type, sn)] 
            for i in input_df.columns.to_list():
                new_col = []
                for el in list(input_df[i]):
                    new_col.append(el*wts[i])
                weighted_df[i] = new_col
            X = np.array(weighted_df)
            X_embedded = TSNE(n_components = 2,
                              learning_rate = 'auto',
                              init='random',
                              perplexity = 30).fit_transform(X)
            tsne_df = pd.DataFrame({'tsne_1': X_embedded[:,0],
                                    'tsne_2': X_embedded[:,1]})
            tsne_df.index = weighted_df.index
            for lbl in lbl_list:
                tsne_df['label'] = lbl
                fig = px.scatter(tsne_df, x="tsne_1", y="tsne_2", color='label', width=500, height=400, title= "%s-%s-%s" % (my_tag, dist_type,sn))
                figs.append(fig)
    return figs

In [ ]:
figs = make_tsne_mixed(["wishart", "podani", "gower"], skmedoids_norm_weights, all_df_enc_norm, [lbls], ['2.1','3.5','4.4'])

In [ ]:
len(figs)

In [ ]:
for fig in figs: fig.show()